## 正则表达式指南

## 概述

正则表达式（Regular expressions，也叫 REs、 regexs 或 regex patterns），本质上是嵌入 Python 内部并通过 [`re`](https://docs.python.org/zh-cn/3/library/re.html#module-re) 模块提供的一种微小的、高度专业化的编程语言。使用这种小语言，你可以为想要匹配的可能字符串编写规则；这些字符串可能是英文句子、邮箱地址、TeX  命令或任何你喜欢的内容。然后，你可以提出诸如“此字符串是否与表达式匹配？”、“字符串中是否存在表达式的匹配项？”之类的问题。你还可以用正则来修改字符串，或以各种方式将其拆分。

正则表达式会被编译成一系列字节码，然后由 C 语言编写的匹配引擎执行。对于高级用途，可能有必要特别注意引擎将如何执行一个给定的正则，并以某种方式写入正则，以生成运行更快的字节码。本文不涉及优化问题，因为这要求你对正则引擎的匹配过程有很好的了解。

正则表达式语言相对较小且受限，因此并非所有可能的字符串处理任务都可以使用正则表达式完成。有些任务尽管*可以*用正则表达式来完成，但表达式会变得非常复杂。这些情况下，最好通过编写 Python 代码来进行处理。也许 Python 代码会比精心设计的正则表达式慢，但它可能更容易理解。

## 简单正则

让我们从最简单的正则表达式开始吧。由于正则表达式是用来操作字符串的，我们将从最常见的任务开始：匹配字符。

关于正则表达式背后的计算机科学的详细解释（确定性和非确定性有限自动机），你可以参考几乎所有关于编写编译器的教科书。


### 匹配字符

大多数字母和符号都会简单地匹配自身。例如，正则表达式 `test` 将会精确地匹配到 `test` 。（你可以启用不区分大小写模式，让这个正则也匹配 `Test` 或 `TEST` ，稍后会详细介绍。）

但该规则有例外。有些字符是特殊的 *元字符（metacharacters）*，并不匹配自身。事实上，它们表示匹配一些非常规的内容，或者通过重复它们或改变它们的含义来影响正则的其他部分。本文的大部分内容都致力于讨论各种元字符及其作用。

这是元字符的完整列表。它们的含义将在本 HOWTO 的其余部分进行讨论。

```
. ^ $ * + ? { } [ ] \ | ( )
```

首先介绍的元字符是 `[` 和 `]` 。这两个元字符用于指定一个字符类，也就是你希望匹配的字符的一个集合。这些字符可以单独地列出，也可以用字符范围来表示（给出两个字符并用 `'-'` 分隔）。例如，`[abc]` 将匹配 `a`、`b`、`c` 之中的任意一个字符；这与 `[a-c]` 相同，后者使用一个范围来表达相同的字符集合。如果只想匹配小写字母，则正则表达式将是 `[a-z]` 。

元字符 (除了 `\`) 在字符类中是不起作用的。 例如，`[akm$]` 将会匹配以下任一字符 `'a'`, `'k'`, `'m'` 或 `'$'`；`'$'` 通常是一个元字符，但在一个字符类中它的特殊性被消除了。

你可以通过对集合 *取反* 来匹配字符类中未列出的字符。方法是把 `'^'` 放在字符类的最开头。 例如，`[^5]` 将匹配除 `'5'` 之外的任何字符。 如果插入符出现在字符类的其他位置，则它没有特殊含义。 例如：`[5^]` 将匹配 `'5'` 或 `'^'`。

也许最重要的元字符是反斜杠，`\` 。 与 Python 字符串字面量一样，反斜杠后面可以跟各种字符来表示各种特殊序列。它还用于转义元字符，以便可以在表达式中匹配元字符本身。例如，如果需要匹配一个 `[` 或 `\` ，可以在其前面加上一个反斜杠来消除它们的特殊含义：`\[` 或 `\\` 。

一些以 `'\'` 开头的特殊序列表示预定义的字符集合，这些字符集通常很有用，例如数字集合、字母集合或非空白字符集合。

让我们举一个例子：`\w` 匹配任何字母数字字符。 如果正则表达式以 bytes 类型表示，`\w` 相当于字符类 `[a-zA-Z0-9_]` 。如果正则表达式是 str 类型，`\w` 将匹配由 [`unicodedata`](https://docs.python.org/zh-cn/3/library/unicodedata.html#module-unicodedata) 模块提供的 Unicode 数据库中标记为字母的所有字符。 通过在编译正则表达式时提供 [`re.ASCII`](https://docs.python.org/zh-cn/3/library/re.html#re.ASCII) 标志，可以在 str 表达式中使用较为狭窄的 `\w` 定义。

以下为特殊序列的不完全列表。 有关 Unicode 字符串正则表达式的序列和扩展类定义的完整列表，参见标准库参考中 [正则表达式语法](https://docs.python.org/zh-cn/3/library/re.html#re-syntax)  的最后一部分 。通常，Unicode 版本的字符类会匹配 Unicode 数据库的相应类别中的任何字符。

- `\d`

  匹配任何十进制数字，等价于字符类 `[0-9]` 。

- `\D`

  匹配任何非数字字符，等价于字符类 `[^0-9]` 。

- `\s`

  匹配任何空白字符，等价于字符类 `[ \t\n\r\f\v]` 。

- `\S`

  匹配任何非空白字符，等价于字符类 `[^ \t\n\r\f\v]` 。

- `\w`

  匹配任何字母与数字字符，等价于字符类 `[a-zA-Z0-9_]` 。

- `\W`

  匹配任何非字母与数字字符，等价于字符类 `[^a-zA-Z0-9_]` 。

这些序列可以包含在字符类中。 例如，`[\s,.]` 是一个匹配任何空白字符、`','` 或 `'.'` 的字符类。

本节的最后一个元字符是 `.` 。 它匹配除换行符之外的任何字符，并且有一个可选模式（ [`re.DOTALL`](https://docs.python.org/zh-cn/3/library/re.html#re.DOTALL) ），在该模式下它甚至可以匹配换行符。 `.` 通常用于你想匹配“任何字符”的场景。

### 重复

能够匹配各种各样的字符集合是正则表达式可以做到的第一件事，而这是字符串方法所不能做到的。但是，如果正则表达式就只有这么一个附加功能，它很难说的上有多大优势。另一个功能是，你可以指定正则的某部分必须重复一定的次数。

我们先来说说重复元字符 `*` 。 `*` 并不是匹配一个字面字符  `'*'` 。实际上，它指定前一个字符可以匹配零次或更多次，而不是只匹配一次。

例如，`ca*t` 将匹配 `'ct'` （ 0 个 `'a'` ）、`'cat'` （ 1 个 `'a'` ）、 `'caaat'` （ 3 个 `'a'` ）等等。

类似 `*` 这样的重复是 *贪婪的* 。当重复正则时，匹配引擎将尝试重复尽可能多的次数。 如果表达式的后续部分不匹配，则匹配引擎将回退并以较少的重复次数再次尝试。

通过一个逐步示例更容易理解这一点。让我们分析一下表达式 `a[bcd]*b` 。 该表达式首先匹配一个字母 `'a'` ，接着匹配字符类 `[bcd]` 中的零个或更多个字母，最后以一个 `'b'` 结尾。 现在想象一下用这个正则来匹配字符串 `'abcbd'` 。

| 步骤 | 匹配    | 说明                                                         |
| ---- | ------- | ------------------------------------------------------------ |
| 1    | `a`     | 正则中的 `a` 匹配成功。                                      |
| 2    | `abcbd` | 引擎尽可能多地匹配 `[bcd]*` ，直至字符串末尾。               |
| 3    | *失败*  | 引擎尝试匹配 `b` ，但是当前位置位于字符串末尾，所以匹配失败。 |
| 4    | `abcb`  | 回退，让 `[bcd]*` 少匹配一个字符。                           |
| 5    | *失败*  | 再次尝试匹配 `b` ， 但是当前位置上的字符是最后一个字符 `'d'` 。 |
| 6    | `abc`   | 再次回退，让 `[bcd]*` 只匹配 `bc` 。                         |
| 6    | `abcb`  | 再次尝试匹配 `b` 。 这一次当前位置的字符是 `'b'` ，所以它成功了。 |

此时正则表达式已经到达了尽头，并且匹配到了 `'abcb'` 。 这个例子演示了匹配引擎一开始会尽其所能地进行匹配，如果没有找到匹配，它将逐步回退并重试正则的剩余部分，如此往复，直至 `[bcd]*` 只匹配零次。如果随后的匹配还是失败了，那么引擎会宣告整个正则表达式与字符串匹配失败。

另一个重复元字符是 `+` ，表示匹配一次或更多次。请注意 `*` 与 `+` 之间的差别。 `*` 表示匹配 *零次* 或更多次，也就是说它所重复的内容是可以完全不出现的。而 `+` 则要求至少出现一次。举一个类似的例子， `ca+t` 可以匹配 `'cat'` （ 1 个 `'a'` ）或 `'caaat'` （ 3 个 `'a'`），但不能匹配 `'ct'` 。

此外还有两个重复操作符或限定符。 问号 `?` 表示匹配一次或零次；你可以认为它把某项内容变成了可选的。 例如，`home-?brew` 可以匹配 `'homebrew'` 或 `'home-brew'`。

最复杂的限定符是 `{m,n}`，其中 *m* 和 *n* 都是十进制整数。 该限定符表示必须至少重复 *m* 次，至多重复 *n* 次。 例如，`a/{1,3}b` 将匹配 `'a/b'`, `'a//b'` 和 `'a///b'`。 它不能匹配 `'ab'`，因为其中没有斜杠，也不能匹配 `'a////b'`，因为其中有四个斜杠。

*m* 和 *n* 不是必填的，缺失的情况下会设定为默认值。缺失 *m* 会解释为最少重复 0 次 ，缺失 *n* 则解释为最多重复无限次。

最简单情况 `{m}` 将与前一项完全匹配 *m* 次。 例如，`a/{2}b` 将只匹配 `'a//b'`。

细心的读者可能会注意到另外三个限定符都可以使用此标记法来表示。 `{0,}` 等同于 `*`, `{1,}` 等同于 `+`, 而 `{0,1}` 等同于 `?`。 在可能的情况下使用 `*`, `+` 或 `?` 会更好，因为它们更为简短易读。

### 使用正则表达式

现在我们已经看了一些简单的正则表达式，那么我们实际在 Python 中是如何使用它们的呢？ re 模块提供了一个正则表达式引擎的接口，可以让你将 REs 编译成对象并用它们来进行匹配。

In [26]:
import re

#### 编译正则表达式
正则表达式被编译成 `RegexObject` 实例，可以为不同的操作提供方法，如模式匹配搜索或字符串替换。

In [27]:
#!python
>>> import re
>>> p = re.compile('ab*')
>>> print(p)

re.compile('ab*')


re.compile() 也接受可选的标志参数，常用来实现不同的特殊功能和语法变更。我们稍后将查看所有可用的设置，但现在只举一个例子：

In [2]:
#!python
>>> import re
>>> p = re.compile('ab*', re.IGNORECASE)

### 反斜杠灾难

如前所述，正则表达式使用反斜杠字符 (`'\'`) 来表示特殊形式或允许使用特殊字符而不调用它们的特殊含义。 这与 Python 在字符串文字中用于相同目的的相同字符的使用相冲突。

假设你想要编写一个与字符串 `\section` 相匹配的正则，它可以在 LaTeX 文件中找到。 要找出在程序代码中写入的内容，请从要匹配的字符串开始。 接下来，您必须通过在反斜杠前面添加反斜杠和其他元字符，从而产生字符串 `\\section`。 必须传递给 [`re.compile()`](https://docs.python.org/zh-cn/3/library/re.html#re.compile) 的结果字符串必须是 `\\section`。 但是，要将其表示为 Python 字符串文字，必须 *再次* 转义两个反斜杠。

| 字符            | 阶段                                                         |
| --------------- | ------------------------------------------------------------ |
| `\section`      | 被匹配的字符串                                               |
| `\\section`     | 为 [`re.compile()`](https://docs.python.org/zh-cn/3/library/re.html#re.compile) 转义的反斜杠 |
| `"\\\\section"` | 为字符串字面转义的反斜杠                                     |

简而言之，要匹配文字反斜杠，必须将 `'\\\\'` 写为正则字符串，因为正则表达式必须是 `\\`，并且每个反斜杠必须表示为 `\\` 在常规Python字符串字面中。 在反复使用反斜杠的正则中，这会导致大量重复的反斜杠，并使得生成的字符串难以理解。

解决方案是使用 Python 的原始字符串表示法来表示正则表达式；反斜杠不以任何特殊的方式处理前缀为 `'r'` 的字符串字面，因此 `r"\n"` 是一个包含 `'\'` 和 `'n'` 的双字符字符串，而 `"\n"` 是一个包含换行符的单字符字符串。 正则表达式通常使用这种原始字符串表示法用 Python 代码编写。

此外，在正则表达式中有效但在 Python 字符串文字中无效的特殊转义序列现在导致 [`DeprecationWarning`](https://docs.python.org/zh-cn/3/library/exceptions.html#DeprecationWarning) 并最终变为 [`SyntaxError`](https://docs.python.org/zh-cn/3/library/exceptions.html#SyntaxError)。 这意味着如果未使用原始字符串表示法或转义反斜杠，序列将无效。

| 常规字符串      | 原始字符串     |
| --------------- | -------------- |
| `"ab*"`         | `r"ab*"`       |
| `"\\\\section"` | `r"\\section"` |
| `"\\w+\\s+\\1"` | `r"\w+\s+\1"`  |

#### 执行匹配

一旦你有了已经编译了的正则表达式的对象，你要用它做什么呢？`RegexObject` 实例有一些方法和属性。这里只显示了最重要的几个，如果要看完整的列表请查阅 Python Library Reference

| 方法/属性  | 作用                                               |
| ---------- | -------------------------------------------------- |
| match()    | 决定 RE 是否在字符串刚开始的位置匹配               |
| search()   | 扫描字符串，找到这个 RE 匹配的位置                 |
| findall()  | 找到 RE 匹配的所有子串，并把它们作为一个列表返回   |
| finditer() | 找到 RE 匹配的所有子串，并把它们作为一个迭代器返回 |


如果没有匹配到的话，match() 和 search() 将返回 None。如果成功的话，就会返回一个 `MatchObject` 实例，其中有这次匹配的信息：它是从哪里开始和结束，它所匹配的子串等等。


你可以用采用人机对话并用 re 模块实验的方式来学习它。如果你有 Tkinter 的话，你也许可以考虑参考一下 Tools/scripts/redemo.py，一个包含在 Python 发行版里的示范程序。


首先，运行 Python 解释器，导入 re 模块并编译一个 RE：

In [10]:
import re
p = re.compile('[a-z]+')
print(p)

re.compile('[a-z]+')


现在，你可以试着用 RE 的 [a-z]+ 去匹配不同的字符串。一个空字符串将根本不能匹配，因为 + 的意思是 “一个或更多的重复次数”。 在这种情况下 match() 将返回 None，因为它使解释器没有输出。你可以明确地打印出 match() 的结果来弄清这一点。

In [11]:
p.match("")
print(p.match(""))

None


现在，让我们试着用它来匹配一个字符串，如 "tempo"。这时，match() 将返回一个 MatchObject。因此你可以将结果保存在变量里以便後面使用。

In [12]:
m = p.match( 'tempo')
print(m)

<re.Match object; span=(0, 5), match='tempo'>


现在你可以查询 `MatchObject` 关于匹配字符串的相关信息了。MatchObject 实例也有几个方法和属性；最重要的那些如下所示：

| 方法/属性 | 作用                                    |
| --------- | --------------------------------------- |
| group()   | 返回被 RE 匹配的字符串                  |
| start()   | 返回匹配开始的位置                      |
| end()     | 返回匹配结束的位置                      |
| span()    | 返回一个元组包含匹配 (开始,结束) 的位置 |


试试这些方法不久就会清楚它们的作用了：

In [14]:
print(m.group())
print((m.start(), m.end()))
print(m.span())

tempo
(0, 5)
(0, 5)


group() 返回 RE 匹配的子串。start() 和 end() 返回匹配开始和结束时的索引。span()  则用单个元组把开始和结束时的索引一起返回。因为匹配方法检查到如果 RE 在字符串开始处开始匹配，那么 start() 将总是为零。然而，  `RegexObject` 实例的 search 方法扫描下面的字符串的话，在这种情况下，匹配开始的位置就也许不是零了。

In [16]:
print(p.match('::: message'))
m = p.search('::: message') 
print(m)
print(m.group())
print(m.span())

None
<re.Match object; span=(4, 11), match='message'>
message
(4, 11)


在实际程序中，最常见的作法是将 `MatchObject` 保存在一个变量里，然後检查它是否为 None，通常如下所示： 

In [18]:
p = re.compile('.abc')
m = p.match( 'string goes here' )
if m:
    print('Match found: ', m.group())
else:
    print('No match')

No match


两个 `RegexObject` 方法返回所有匹配模式的子串。findall()返回一个匹配字符串行表： 

In [20]:
import re
p = re.compile('\d+')
print(p.findall('12 drummers drumming, 11 pipers piping, 10 lords a-leaping'))

['12', '11', '10']


findall() 在它返回结果时不得不创建一个列表。在 Python 2.2中，也可以用 finditer() 方法。 

In [21]:
iterator = p.finditer('12 drummers drumming, 11 ... 10 ...')
print(iterator)
for match in iterator:
    print(match.span())

(0, 2)
(22, 24)
(29, 31)


#### 模块级函数

你不一定要产生一个 `RegexObject` 对象然后再调用它的方法；re 模块也提供了顶级函数调用如  match()、search()、sub() 等等。这些函数使用 RE 字符串作为第一个参数，而后面的参数则与相应 `RegexObject`  的方法参数相同，返回则要么是 None 要么就是一个 `MatchObject` 的实例。

In [22]:
#!python
>>> import re
>>> print(re.match(r'From\s+', 'Fromage amk'))
>>> re.match(r'From\s+', 'From amk Thu May 14 19:12:10 1998')

None


<re.Match object; span=(0, 5), match='From '>

In [ ]:
 在引擎内部，这些函数简单地产生一个 RegexOject 并在其上调用相应的方法。它们也在缓存里保存编译后的对象，因此在将来调用用到相同 RE 时就会更快。 

#### 编译标志

编译标志让你可以修改正则表达式的一些运行方式。在 re 模块中标志可以使用两个名字，一个是全名如  IGNORECASE，一个是缩写，一字母形式如 I。（如果你熟悉 Perl 的模式修改，一字母形式使用同样的字母；例如  re.VERBOSE的缩写形式是 re.X。）多个标志可以通过按位 OR-ing 它们来指定。如 re.I | re.M 被设置成 I 和 M  标志：


 这有个可用标志表，对每个标志后面都有详细的说明。

| 标志          | 含义                                                 |
| ------------- | ---------------------------------------------------- |
| DOTALL, S     | 使 . 匹配包括换行在内的所有字符                      |
| IGNORECASE, I | 使匹配对大小写不敏感                                 |
| LOCALE, L     | 做本地化识别（locale-aware）匹配                     |
| MULTILINE, M  | 多行匹配，影响 ^ 和 $                                |
| VERBOSE, X    | 能够使用 REs 的 verbose 状态，使之被组织得更清晰易懂 |

**I**
 **IGNORECASE**

使匹配对大小写不敏感；字符类和字符串匹配字母时忽略大小写。举个例子，[A-Z]也可以匹配小写字母，Spam 可以匹配 "Spam", "spam", 或 "spAM"。这个小写字母并不考虑当前位置。

**L**
 **LOCALE**

影响 \w, \W, \b, 和 \B，这取决于当前的本地化设置。

locales 是 C 语言库中的一项功能，是用来为需要考虑不同语言的编程提供帮助的。举个例子，如果你正在处理法文文本，你想用  \w+ 来匹配文字，但 \w 只匹配字符类 [A-Za-z]；它并不能匹配 "é" 或  "ç"。如果你的系统配置适当且本地化设置为法语，那么内部的 C 函数将告诉程序 "é" 也应该被认为是一个字母。当在编译正则表达式时使用  LOCALE 标志会得到用这些 C 函数来处理 \w 后的编译对象；这会更慢，但也会象你希望的那样可以用 \w+ 来匹配法文文本。

**M**
 **MULTILINE**


 (此时 ^ 和 $ 不会被解释; 它们将在 4.1 节被介绍.)


 使用 "^" 只匹配字符串的开始，而 $ 则只匹配字符串的结尾和直接在换行前（如果有的话）的字符串结尾。当本标志指定后， "^" 匹配字符串的开始和字符串中每行的开始。同样的， $ 元字符匹配字符串结尾和字符串中每行的结尾（直接在每个换行之前）。

**S**
 **DOTALL**

使 "." 特殊字符完全匹配任何字符，包括换行；没有这个标志， "." 匹配除了换行外的任何字符。

**X**
 **VERBOSE**


 该标志通过给予你更灵活的格式以便你将正则表达式写得更易于理解。当该标志被指定时，在 RE  字符串中的空白符被忽略，除非该空白符在字符类中或在反斜杠之后；这可以让你更清晰地组织和缩进 RE。它也可以允许你将注释写入  RE，这些注释会被引擎忽略；注释用 "#"号 来标识，不过该符号不能在字符串或反斜杠之后。

举个例子，这里有一个使用 re.VERBOSE 的 RE；看看读它轻松了多少？

In [26]:
charref = re.compile(r"""&[[]]		   # Start of a numeric entity reference|||here has wrong.i can't fix
(
[0-9]+[^0-9]      # Decimal form
| 0[0-7]+[^0-7]   # Octal form
| x[0-9a-fA-F]+[^0-9a-fA-F] # Hexadecimal form
)
""", re.VERBOSE)

In [27]:
charref = re.compile("&#([0-9]+[^0-9]"
"|0[0-7]+[^0-7]"
"|x[0-9a-fA-F]+[^0-9a-fA-F])")

在上面的例子里，Python 的字符串自动连接可以用来将 RE 分成更小的部分，但它比用 re.VERBOSE 标志时更难懂 

## 更多模式能力

### 更多元字符

要讨论的其余一些元字符是 *零宽度断言* 。 它们不会使解析引擎在字符串中前进一个字符；相反，它们根本不占用任何字符，只是成功或失败。例如，`\b` 是一个断言，指明当前位置位于字边界；这个位置根本不会被 `\b` 改变。这意味着永远不应重复零宽度断言，因为如果它们在给定位置匹配一次，它们显然可以无限次匹配。

```
|
```

或者“or”运算符。 如果 *A* 和 *B* 是正则表达式，`A|B` 将匹配任何与 *A* 或 *B* 匹配的字符串。 `|` 具有非常低的优先级，以便在交替使用多字符字符串时使其合理地工作。 `Crow|Servo` 将匹配 `'Crow'` 或 `'Servo'`，而不是 `'Cro'`、`'w'` 或 `'S'` 和 `'ervo'`。

要匹配字面 `'|'`，请使用 `\|`，或将其括在字符类中，如 `[|]`。

```
^
```

在行的开头匹配。 除非设置了 [`MULTILINE`](https://docs.python.org/zh-cn/3/library/re.html#re.MULTILINE) 标志，否则只会在字符串的开头匹配。 在 [`MULTILINE`](https://docs.python.org/zh-cn/3/library/re.html#re.MULTILINE) 模式下，这也在字符串中的每个换行符后立即匹配。

例如，如果你希望仅在行的开头匹配单词 `From`，则要使用的正则 `^From`。:

In [3]:
print(re.search('^From', 'From Here to Eternity'))  
print(re.search('^From', 'Reciting From Memory'))

<re.Match object; span=(0, 4), match='From'>
None


要匹配字面 `'^'`，使用 `\^`。

```
$
```

匹配行的末尾，定义为字符串的结尾，或者后跟换行符的任何位置。:

In [4]:
print(re.search('}$', '{block}'))  
print(re.search('}$', '{block} '))
print(re.search('}$', '{block}\n'))  

<re.Match object; span=(6, 7), match='}'>
None
<re.Match object; span=(6, 7), match='}'>


以匹配字面 `'$'`，使用 `\$` 或者将其包裹在一个字符类中，例如 `[$]`。

```
\A
```

仅匹配字符串的开头。 当不在 [`MULTILINE`](https://docs.python.org/zh-cn/3/library/re.html#re.MULTILINE) 模式时，`\A` 和 `^` 实际上是相同的。 在 [`MULTILINE`](https://docs.python.org/zh-cn/3/library/re.html#re.MULTILINE) 模式中，它们是不同的: `\A` 仍然只在字符串的开头匹配，但 `^` 可以匹配在换行符之后的字符串内的任何位置。

```
\Z
```

只匹配字符串尾。

```
\b
```

字边界。 这是一个零宽度断言，仅在单词的开头或结尾处匹配。 单词被定义为一个字母数字字符序列，因此单词的结尾由空格或非字母数字字符表示。

以下示例仅当它是一个完整的单词时匹配 `class`；当它包含在另一个单词中时将不会匹配。

In [5]:
p = re.compile(r'\bclass\b')
print(p.search('no class at all'))
print(p.search('the declassified algorithm'))
print(p.search('one subclass is'))

<re.Match object; span=(3, 8), match='class'>
None
None


使用这个特殊序列时，你应该记住两个细微之处。 首先，这是 Python 的字符串文字和正则表达式序列之间最严重的冲突。 在 Python 的字符串文字中，`\b` 是退格字符，ASCII 值为8。 如果你没有使用原始字符串，那么 Python 会将 `\b` 转换为退格，你的正则不会按照你的预期匹配。 以下示例与我们之前的正则看起来相同，但省略了正则字符串前面的 `'r'`。:

In [6]:
p = re.compile('\bclass\b')
print(p.search('no class at all'))
print(p.search('\b' + 'class' + '\b'))

None
<re.Match object; span=(0, 7), match='\x08class\x08'>


其次，在一个字符类中，这个断言没有用处，`\b` 表示退格字符，以便与 Python 的字符串文字兼容。

```
\B
```

另一个零宽度断言，这与 `\b` 相反，仅在当前位置不在字边界时才匹配。

### 分组

通常，你需要获取更多信息，而不仅仅是正则是否匹配。 正则表达式通常用于通过将正则分成几个子组来解析字符串，这些子组匹配不同的感兴趣组件。 例如，RFC-822 标题行分为标题名称和值，用 `':'` 分隔，如下所示：

```
From: author@example.com
User-Agent: Thunderbird 1.5.0.9 (X11/20061227)
MIME-Version: 1.0
To: editor@example.com
```

这可以通过编写与整个标题行匹配的正则表达式来处理，并且具有与标题名称匹配的一个组，以及与标题的值匹配的另一个组。

分组是用 `'('`, `')'` 元字符来标记的。 `'('` 和 `')'` 与它们在数学表达式中的含义基本一致；它们会将所包含的表达式合为一组，并且你可以使用限定符例如 `*`, `+`, `?`, 或 `{m,n}` 来重复一个分组的内容。 举例来说，`(ab)*` 将匹配 `ab` 的零次或多次重复。

In [7]:
p = re.compile('(ab)*')
print(p.match('ababababab').span())

(0, 10)


用 `'('`，`')'` 表示的组也捕获它们匹配的文本的起始和结束索引；这可以通过将参数传递给 [`group()`](https://docs.python.org/zh-cn/3/library/re.html#re.Match.group)、[`start()`](https://docs.python.org/zh-cn/3/library/re.html#re.Match.start)、[`end()`](https://docs.python.org/zh-cn/3/library/re.html#re.Match.end) 以及 [`span()`](https://docs.python.org/zh-cn/3/library/re.html#re.Match.span)。 组从 0 开始编号。组 0 始终存在；它表示整个正则，所以 [匹配对象](https://docs.python.org/zh-cn/3/library/re.html#match-objects) 方法都将组 0 作为默认参数。 稍后我们将看到如何表达不捕获它们匹配的文本范围的组。:

In [9]:
p = re.compile('(a)b')
m = p.match('ab')
print(m.group())
print(m.group(0))

ab
ab


子组从左到右编号，从 1 向上编号。 组可以嵌套；要确定编号，只需计算从左到右的左括号字符。:

In [12]:
p = re.compile('(a(b)c)d')
m = p.match('abcd')
print(m.group(0))
print(m.group(1))
print(m.group(2))

abcd
abc
b


[`group()`](https://docs.python.org/zh-cn/3/library/re.html#re.Match.group) 可以一次传递多个组号，在这种情况下，它将返回一个包含这些组的相应值的元组。:

In [13]:
m.group(2,1,2)

('b', 'abc', 'b')

[`groups()`](https://docs.python.org/zh-cn/3/library/re.html#re.Match.groups) 方法返回一个元组，其中包含所有子组的字符串，从1到最后一个子组。:



In [14]:
m.groups()

('abc', 'b')

模式中的后向引用允许你指定还必须在字符串中的当前位置找到先前捕获组的内容。 例如，如果可以在当前位置找到组 1 的确切内容，则 `\1` 将成功，否则将失败。 请记住，Python 的字符串文字也使用反斜杠后跟数字以允许在字符串中包含任意字符，因此正则中引入反向引用时务必使用原始字符串。

例如，以下正则检测字符串中重复的单词。:

In [15]:
p = re.compile(r'\b(\w+)\s+\1\b')
p.search('Paris in the the spring').group()

'the the'

像这样的后向引用通常不仅仅用于搜索字符串 —— 很少有文本格式以这种方式重复数据 —— 但是你很快就会发现它们在执行字符串替换时 *非常* 有用。

### 非捕获和命名组

精心设计的正则可以使用许多组，既可以捕获感兴趣的子串，也可以对正则本身进行分组和构建。 在复杂的正则中，很难跟踪组号。 有两个功能可以帮助解决这个问题。 它们都使用常用语法进行正则表达式扩展，因此我们首先看一下。

Perl 开发人员选择的解决方案是使用 `(?...)` 作为扩展语法。 括号后面紧跟 `?` 是一个语法错误，因为 `?` 没有什么可重复的，所以这样并不会带来任何兼容性问题。 紧跟在 `?` 之后的字符表示正在使用的扩展语法，所以 `(?=foo)` 是一种语法（一个前视断言）和 `(?:foo)` 是另一种语法（ 包含子表达式 `foo` 的非捕获组）。

Python 支持一些 Perl 的扩展，并增加了新的扩展语法用于 Perl 的扩展语法。 如果在问号之后的第一个字符为 `P`，即表明其为 Python 专属的扩展。

现在我们已经了解了一般的扩展语法，我们可以回到简化复杂正则中组处理的功能。

有时你会想要使用组来表示正则表达式的一部分，但是对检索组的内容不感兴趣。 你可以通过使用非捕获组来显式表达这个事实: `(?:...)`，你可以用任何其他正则表达式替换 `...`。:

In [20]:
m = re.match("([abc])+", "abc")
print(m.groups())

m = re.match("(?:[abc])+", "abc")
print(m.groups())

('c',)
()


除了你无法检索组匹配内容的事实外，非捕获组的行为与捕获组完全相同；你可以在里面放任何东西，用重复元字符重复它，比如 `*`，然后把它嵌入其他组（捕获或不捕获）。 `(?:...)` 在修改现有模式时特别有用，因为你可以添加新组而不更改所有其他组的编号方式。 值得一提的是，捕获和非捕获组之间的搜索没有性能差异；两种形式没有一种更快。

更重要的功能是命名组：不是通过数字引用它们，而是可以通过名称引用组。

命名组的语法是Python特定的扩展之一: `(?P<name>...)`。 *name* 显然是该组的名称。 命名组的行为与捕获组完全相同，并且还将名称与组关联。 处理捕获组的 [匹配对象](https://docs.python.org/zh-cn/3/library/re.html#match-objects) 方法都接受按编号引用组的整数或包含所需组名的字符串。 命名组仍然是给定的数字，因此你可以通过两种方式检索有关组的信息:

In [21]:
p = re.compile(r'(?P<word>\b\w+\b)')
m = p.search( '(((( Lots of punctuation )))' )
print(m.group('word'))
print(m.group(1))

Lots
Lots


此外，你可以通过 [`groupdict()`](https://docs.python.org/zh-cn/3/library/re.html#re.Match.groupdict) 将命名分组提取为一个字典:

In [22]:
m = re.match(r'(?P<first>\w+) (?P<last>\w+)', 'Jane Doe')
m.groupdict()

{'first': 'Jane', 'last': 'Doe'}

命名分组很方便因为它们让你可以使用容易记忆的名称，而不必记忆数字。 下面是一个来自 [`imaplib`](https://docs.python.org/zh-cn/3/library/imaplib.html#module-imaplib) 模块的正则表达式示例:

In [23]:
InternalDate = re.compile(r'INTERNALDATE "'
        r'(?P<day>[ 123][0-9])-(?P<mon>[A-Z][a-z][a-z])-'
        r'(?P<year>[0-9][0-9][0-9][0-9])'
        r' (?P<hour>[0-9][0-9]):(?P<min>[0-9][0-9]):(?P<sec>[0-9][0-9])'
        r' (?P<zonen>[-+])(?P<zoneh>[0-9][0-9])(?P<zonem>[0-9][0-9])'
        r'"')

检索 `m.group('zonem')` 显然要容易得多，而不必记住检索第 9 组。

表达式中的后向引用语法，例如 `(...)\1`，指的是组的编号。 当然有一种变体使用组名而不是数字。 这是另一个 Python 扩展: `(?P=name)` 表示在当前点再次匹配名为 *name* 的组的内容。 用于查找重复单词的正则表达式，`\b(\w+)\s+\1\b` 也可以写为 `\b(?P<word>\w+)\s+(?P=word)\b`:

In [24]:
p = re.compile(r'\b(?P<word>\w+)\s+(?P=word)\b')
p.search('Paris in the the spring').group()

'the the'

### 前视断言

另一个零宽断言是前视断言。 前视断言有肯定型和否定型两种形式，如下所示：

- `(?=…)`

  肯定型前视断言。如果内部的表达式（这里用 `...` 来表示）在当前位置可以匹配，则匹配成功，否则匹配失败。 但是，内部表达式尝试匹配之后，正则引擎并不会向前推进；正则表达式的其余部分依然会在断言开始的地方尝试匹配。

- `(?!…)`

  否定型前视断言。 与肯定型断言正好相反，如果内部表达式在字符串中的当前位置 **不** 匹配，则成功。

更具体一些，来看一个前视的实用案例。 考虑用一个简单的表达式来匹配文件名并将其拆分为基本名称和扩展名，以 `.` 分隔。 例如，在 `news.rc` 中，`news` 是基本名称，`rc` 是文件名的扩展名。

与此匹配的模式非常简单：

```
.*[.].*$
```

请注意，`.` 需要特别处理，因为它是元字符，所以它在字符类中只能匹配特定字符。 还要注意尾随的 `$`；添加此项以确保扩展名中的所有其余字符串都必须包含在扩展名中。 这个正则表达式匹配 `foo.bar`、`autoexec.bat`、`sendmail.cf` 和 `printers.conf`。

现在，考虑使更复杂一点的问题；如果你想匹配扩展名不是 `bat` 的文件名怎么办？ 一些错误的尝试：

`.*[.][^b].*$` 上面的第一次尝试试图通过要求扩展名的第一个字符不是 `b` 来排除 `bat`。 这是错误的，因为模式也与 `foo.bar` 不匹配。

```
.*[.]([^b]..|.[^a].|..[^t])$
```

当你尝试通过要求以下一种情况匹配来修补第一个解决方案时，表达式变得更加混乱：扩展的第一个字符不是 `b`。 第二个字符不 `a`；或者第三个字符不是 `t`。 这接受 `foo.bar` 并拒绝 `autoexec.bat`，但它需要三个字母的扩展名，并且不接受带有两个字母扩展名的文件名，例如 `sendmail.cf`。 为了解决这个问题，我们会再次使模式复杂化。

```
.*[.]([^b].?.?|.[^a]?.?|..?[^t]?)$
```

在第三次尝试中，第二个和第三个字母都是可选的，以便允许匹配的扩展名短于三个字符，例如 `sendmail.cf`。

模式现在变得非常复杂，这使得它难以阅读和理解。 更糟糕的是，如果问题发生变化并且你想要将 `bat` 和 `exe` 排除为扩展，那么该模式将变得更加复杂和混乱。

否定型前视可以解决所有这些困扰：

`.*[.](?!bat$)[^.]*$` 否定型前视意味着：如果表达式 `bat` 在当前位置不能匹配，则可以接着尝试正则表达式的其余部分；如果 `bat$` 能匹配，则整个正则表达式将匹配失败。尾随的 `$` 是必需的，以确保可以匹配到像 `sample.batch` 这样以 `bat` 开头的文件名。当文件名中有多个点号时， `[^.]*` 可以确保表达式依然有效。

现在很容易排除另一个文件扩展名；只需在断言中添加它作为替代。  以下模块排除以 `bat` 或 `exe`:

```
.*[.](?!bat$|exe$)[^.]*$
```

## 修改字符串

到目前为止，我们只是针对静态字符串执行搜索。 正则表达式通常也用于以各种方式修改字符串，使用以下模式方法：

| 方法 / 属性 | 目的                                                 |
| ----------- | ---------------------------------------------------- |
| `split()`   | 将字符串拆分为一个列表，在正则匹配的任何地方将其拆分 |
| `sub()`     | 找到正则匹配的所有子字符串，并用不同的字符串替换它们 |
| `subn()`    | 与 `sub()` 相同，但返回新字符串和替换次数            |

### 分割字符串

模式的 [`split()`](https://docs.python.org/zh-cn/3/library/re.html#re.Pattern.split) 方法在正则匹配的任何地方拆分字符串，返回一个片段列表。 它类似于 [`split()`](https://docs.python.org/zh-cn/3/library/stdtypes.html#str.split) 字符串方法，但在分隔符的分隔符中提供了更多的通用性；字符串的 `split()` 仅支持按空格或固定字符串进行拆分。 正如你所期望的那样，还有一个模块级 [`re.split()`](https://docs.python.org/zh-cn/3/library/re.html#re.split) 函数。

- .split(*string*[, *maxsplit=0*])

  通过正则表达式的匹配拆分 *字符串*。 如果在正则中使用捕获括号，则它们的内容也将作为结果列表的一部分返回。 如果 *maxsplit* 非零，则最多执行 *maxsplit* 次拆分。

你可以通过传递 *maxsplit* 的值来限制分割的数量。 当 *maxsplit* 非零时，将最多进行 *maxsplit* 次拆分，并且字符串的其余部分将作为列表的最后一个元素返回。 在以下示例中，分隔符是任何非字母数字字符序列。:


In [28]:
p = re.compile(r'\W+')
print(p.split('This is a test, short and sweet, of split().'))
print(p.split('This is a test, short and sweet, of split().', 3))

['This', 'is', 'a', 'test', 'short', 'and', 'sweet', 'of', 'split', '']
['This', 'is', 'a', 'test, short and sweet, of split().']


有时你不仅对分隔符之间的文本感兴趣，而且还需要知道分隔符是什么。 如果在正则中使用捕获括号，则它们的值也将作为列表的一部分返回。 比较以下调用: